# Introduction to DataFrames
**[Bogumił Kamiński](http://bogumilkaminski.pl/about/), July 17, 2021**

In [1]:
using DataFrames

## Split-apply-combine

### Grouping a data frame

In [2]:
x = DataFrame(id=[1,2,3,4,1,2,3,4], id2=[1,2,1,2,1,2,1,2], v=rand(8))

,id,id2,v
,Int64,Int64,Float64
1,1,1,0.225388
2,2,2,0.385866
3,3,1,0.349788
4,4,2,0.242284
5,1,1,0.345011
6,2,2,0.0225686
7,3,1,0.282489
8,4,2,0.986092


In [3]:
groupby(x, :id)

,id,id2,v
,Int64,Int64,Float64
1,1,1,0.225388
2,1,1,0.345011
,id,id2,v
,Int64,Int64,Float64
1,4,2,0.242284
2,4,2,0.986092


In [4]:
groupby(x, [])

,id,id2,v
,Int64,Int64,Float64
1,1,1,0.225388
2,2,2,0.385866
3,3,1,0.349788
4,4,2,0.242284
5,1,1,0.345011
6,2,2,0.0225686
7,3,1,0.282489
8,4,2,0.986092


In [5]:
gx2 = groupby(x, [:id, :id2])

,id,id2,v
,Int64,Int64,Float64
1,1,1,0.225388
2,1,1,0.345011
,id,id2,v
,Int64,Int64,Float64
1,4,2,0.242284
2,4,2,0.986092


In [6]:
parent(gx2) # get the parent DataFrame 

,id,id2,v
,Int64,Int64,Float64
1,1,1,0.225388
2,2,2,0.385866
3,3,1,0.349788
4,4,2,0.242284
5,1,1,0.345011
6,2,2,0.0225686
7,3,1,0.282489
8,4,2,0.986092


In [7]:
vcat(gx2...) # back to the DataFrame, but in a different order of rows than the original

,id,id2,v
,Int64,Int64,Float64
1,1,1,0.225388
2,1,1,0.345011
3,2,2,0.385866
4,2,2,0.0225686
5,3,1,0.349788
6,3,1,0.282489
7,4,2,0.242284
8,4,2,0.986092


In [8]:
DataFrame(gx2) # the same

,id,id2,v
,Int64,Int64,Float64
1,1,1,0.225388
2,1,1,0.345011
3,2,2,0.385866
4,2,2,0.0225686
5,3,1,0.349788
6,3,1,0.282489
7,4,2,0.242284
8,4,2,0.986092


In [9]:
DataFrame(gx2, keepkeys=false) # drop grouping columns when creating a data frame

,v
,Float64
1,0.225388
2,0.345011
3,0.385866
4,0.0225686
5,0.349788
6,0.282489
7,0.242284
8,0.986092


In [10]:
groupcols(gx2) # vector of names of grouping variables

2-element Vector{Symbol}:
 :id
 :id2

In [11]:
valuecols(gx2) # and non-grouping variables

1-element Vector{Symbol}:
 :v

In [12]:
groupindices(gx2) # group indices in parent(gx2)

8-element Vector{Union{Missing, Int64}}:
 1
 2
 3
 4
 1
 2
 3
 4

In [13]:
kgx2 = keys(gx2)

4-element DataFrames.GroupKeys{GroupedDataFrame{DataFrame}}:
 GroupKey: (id = 1, id2 = 1)
 GroupKey: (id = 2, id2 = 2)
 GroupKey: (id = 3, id2 = 1)
 GroupKey: (id = 4, id2 = 2)

You can index into a `GroupedDataFrame` like to a vector or to a dictionary.
The second form acceps `GroupKey`, `NamedTuple` or a `Tuple`

In [14]:
gx2

,id,id2,v
,Int64,Int64,Float64
1,1,1,0.225388
2,1,1,0.345011
,id,id2,v
,Int64,Int64,Float64
1,4,2,0.242284
2,4,2,0.986092


In [15]:
k = keys(gx2)[1]

GroupKey: (id = 1, id2 = 1)

In [16]:
ntk = NamedTuple(k)

(id = 1, id2 = 1)

In [17]:
tk = Tuple(k)

(1, 1)

the operations below produce the same result and are fast

In [18]:
gx2[1]

,id,id2,v
,Int64,Int64,Float64
1,1,1,0.225388
2,1,1,0.345011


In [19]:
gx2[k]

,id,id2,v
,Int64,Int64,Float64
1,1,1,0.225388
2,1,1,0.345011


In [20]:
gx2[ntk]

,id,id2,v
,Int64,Int64,Float64
1,1,1,0.225388
2,1,1,0.345011


In [21]:
gx2[tk]

,id,id2,v
,Int64,Int64,Float64
1,1,1,0.225388
2,1,1,0.345011


handling missing values

In [22]:
x = DataFrame(id = [missing, 5, 1, 3, missing], x = 1:5)

,id,x
,Int64?,Int64
1,missing,1
2,5,2
3,1,3
4,3,4
5,missing,5


In [23]:
groupby(x, :id) # by default groups include mising values and are not sorted

,id,x
,Int64?,Int64
1,1,3
,id,x
,Int64?,Int64
1,missing,1
2,missing,5


In [24]:
groupby(x, :id, sort=true, skipmissing=true) # but we can change it

,id,x
,Int64?,Int64
1,1,3
,id,x
,Int64?,Int64
1,5,2


### Performing transformations by group using `combine`, `select`, `select!`, `transform`, and `transform!`

In [25]:
using Statistics
using Chain

In [26]:
ENV["LINES"] = 15 # reduce the number of rows in the output

15

In [27]:
x = DataFrame(id=rand('a':'d', 100), v=rand(100))

,id,v
,Char,Float64
1,b,0.67658
2,c,0.0860583
3,d,0.68343
4,a,0.80303
5,a,0.464613
6,d,0.271104
7,a,0.494958
8,d,0.976484
9,b,0.0256035


In [28]:
# apply a function to each group of a data frame
# combine keeps as many rows as are returned from the function
@chain x begin
    groupby(:id)
    combine(:v => mean)
end

,id,v_mean
,Char,Float64
1,b,0.543869
2,c,0.441219
3,d,0.554413
4,a,0.407734


In [29]:
x.id2 = axes(x, 1)

Base.OneTo(100)

In [30]:
# select and transform keep as many rows as are in the source data frame and in correct order
# additionally transform keeps all columns from the source
@chain x begin
    groupby(:id)
    transform(:v => mean)
end

,id,v,id2,v_mean
,Char,Float64,Int64,Float64
1,b,0.67658,1,0.543869
2,c,0.0860583,2,0.441219
3,d,0.68343,3,0.554413
4,a,0.80303,4,0.407734
5,a,0.464613,5,0.407734
6,d,0.271104,6,0.554413
7,a,0.494958,7,0.407734
8,d,0.976484,8,0.554413
9,b,0.0256035,9,0.543869


In [31]:
# note that combine reorders rows by group of GroupedDataFrame
@chain x begin
    groupby(:id)
    combine(:id2, :v => mean)
end

,id,id2,v_mean
,Char,Int64,Float64
1,b,1,0.543869
2,b,9,0.543869
3,b,10,0.543869
4,b,13,0.543869
5,b,14,0.543869
6,b,16,0.543869
7,b,24,0.543869
8,b,26,0.543869
9,b,27,0.543869


In [32]:
# we give a custom name for the result column
@chain x begin
    groupby(:id)
    combine(:v => mean => :res)
end

,id,res
,Char,Float64
1,b,0.543869
2,c,0.441219
3,d,0.554413
4,a,0.407734


In [33]:
# you can have multiple operations
@chain x begin
    groupby(:id)
    combine(:v => mean => :res1, :v => sum => :res2, nrow => :n)
end

,id,res1,res2,n
,Char,Float64,Float64,Int64
1,b,0.543869,14.6845,27
2,c,0.441219,11.9129,27
3,d,0.554413,14.4147,26
4,a,0.407734,8.15469,20


Additional notes:
* `select!` and `transform!` perform operations in-place
* The general syntax for transformation is `source_columns => function => target_column`
* if you pass multiple columns to a function they are treated as positional arguments
* `ByRow` and `AsTable` work exactly like discussed for operations on data frames in 05_columns.ipynb
* you can automatically groupby again the result of `combine`, `select` etc. by passing `ungroup=false` keyword argument to them
* similarly `keepkeys` keyword argument allows you to drop grouping columns from the resulting data frame

It is also allowed to pass a function to all these functions (also - as a special case, as a first argument). In this case the return value can be a table. In particular it allows for an easy dropping of groups if you return an empty table from the function.

If you pass a function you can use a `do` block syntax. In case of passing a function it gets a `SubDataFrame` as its argument.

Here is an example:

In [34]:
combine(groupby(x, :id)) do sdf
    n = nrow(sdf)
    n < 25 ? DataFrame() : DataFrame(n=n) # drop groups with low number of rows
end

,id,n
,Char,Int64
1,b,27
2,c,27
3,d,26


You can also produce multiple columns in a single operation, e.g.:

In [35]:
df = DataFrame(id=[1,1,2,2], val=[1,2,3,4])

,id,val
,Int64,Int64
1,1,1
2,1,2
3,2,3
4,2,4


In [36]:
@chain df begin
    groupby(:id)
    combine(:val => (x -> [x]) => AsTable)
end

,id,x1,x2
,Int64,Int64,Int64
1,1,1,2
2,2,3,4


In [37]:
@chain df begin
    groupby(:id)
    combine(:val => (x -> [x]) => [:c1, :c2])
end

,id,c1,c2
,Int64,Int64,Int64
1,1,1,2
2,2,3,4


It is easy to unnest the column into multiple columns, e.g.

In [38]:
df = DataFrame(a=[(p=1, q=2), (p=3, q=4)])

,a
,NamedTu…
1,"(p = 1, q = 2)"
2,"(p = 3, q = 4)"


In [39]:
select(df, :a => AsTable)

,p,q
,Int64,Int64
1,1,2
2,3,4


In [40]:
df = DataFrame(a=[[1, 2], [3, 4]])

,a
,Array…
1,"[1, 2]"
2,"[3, 4]"


In [41]:
select(df, :a => AsTable) # automatic column names generated

,x1,x2
,Int64,Int64
1,1,2
2,3,4


In [42]:
select(df, :a => [:C1, :C2]) # custom column names generated

,C1,C2
,Int64,Int64
1,1,2
2,3,4


Finally, observe that one can conveniently apply multiple transformations using broadcasting:

In [43]:
df = DataFrame(id=repeat(1:10, 10), x1=1:100, x2=101:200)

,id,x1,x2
,Int64,Int64,Int64
1,1,1,101
2,2,2,102
3,3,3,103
4,4,4,104
5,5,5,105
6,6,6,106
7,7,7,107
8,8,8,108
9,9,9,109


In [44]:
@chain df begin
    groupby(:id)
    combine([:x1, :x2] .=> minimum)
end

,id,x1_minimum,x2_minimum
,Int64,Int64,Int64
1,1,1,101
2,2,2,102
3,3,3,103
4,4,4,104
5,5,5,105
6,6,6,106
7,7,7,107
8,8,8,108
9,9,9,109


In [45]:
@chain df begin
    groupby(:id)
    combine([:x1, :x2] .=> [minimum maximum])
end

,id,x1_minimum,x2_minimum,x1_maximum,x2_maximum
,Int64,Int64,Int64,Int64,Int64
1,1,1,101,91,191
2,2,2,102,92,192
3,3,3,103,93,193
4,4,4,104,94,194
5,5,5,105,95,195
6,6,6,106,96,196
7,7,7,107,97,197
8,8,8,108,98,198
9,9,9,109,99,199


### Aggregation of a data frame using `mapcols`

In [46]:
x = DataFrame(rand(10, 10), :auto)

,x1,x2,x3,x4,x5,x6,x7,x8
,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64
1,0.991035,0.465052,0.380187,0.960458,0.313886,0.132266,0.976021,0.808724
2,0.683473,0.840802,0.0186848,0.348605,0.945677,0.605405,0.461895,0.00702123
3,0.650546,0.414467,0.877956,0.59157,0.265558,0.566195,0.0079046,0.608079
4,0.268317,0.319089,0.609955,0.902323,0.770762,0.991544,0.0910924,0.792966
5,0.652427,0.765766,0.49364,0.113245,0.230824,0.229971,0.00935943,0.212565
6,0.722184,0.732389,0.502374,0.717295,0.615377,0.982335,0.747881,0.821812
7,0.535124,0.806856,0.251819,0.641313,0.223733,0.743746,0.775601,0.784478
8,0.581566,0.695011,0.654364,0.256979,0.744593,0.110663,0.675994,0.761357
9,0.726788,0.294234,0.429835,0.323258,0.0581465,0.996876,0.776092,0.323629


In [47]:
mapcols(mean, x)

,x1,x2,x3,x4,x5,x6,x7,x8,x9
,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64
1,0.641407,0.537062,0.44643,0.54511,0.468423,0.550001,0.453471,0.607353,0.53172


### Mapping rows and columns using `eachcol` and `eachrow`

In [48]:
map(mean, eachcol(x)) # map a function over each column and return a vector

10-element Vector{Float64}:
 0.641406543169046
 0.5370624631625194
 0.4464296406215696
 0.5451097503977332
 0.4684234683450052
 0.5500008752555339
 0.4534707039003424
 0.6073533919772265
 0.5317198784576151
 0.6739648483458158

In [49]:
# an iteration returns a Pair with column name and values
foreach(c -> println(c[1], ": ", mean(c[2])), pairs(eachcol(x)))

x1: 0.641406543169046
x2: 0.5370624631625194
x3: 0.4464296406215696
x4: 0.5451097503977332
x5: 0.4684234683450052
x6: 0.5500008752555339
x7: 0.4534707039003424
x8: 0.6073533919772265
x9: 0.5317198784576151
x10: 0.6739648483458158


In [50]:
# now the returned value is DataFrameRow which works as a NamedTuple but is a view to a parent DataFrame
map(r -> r.x1/r.x2, eachrow(x))

10-element Vector{Float64}:
  2.1310198464146426
  0.8128813289171303
  1.5695960635127038
  0.8408842693325757
  0.8519929685434499
  0.9860667946431837
  0.663221576003684
  0.8367722572181535
  2.470105201563278
 16.304585899881697

In [51]:
# it prints like a data frame, only the caption is different so that you know the type of the object
er = eachrow(x)

,x1,x2,x3,x4,x5,x6,x7,x8
,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64
1,0.991035,0.465052,0.380187,0.960458,0.313886,0.132266,0.976021,0.808724
2,0.683473,0.840802,0.0186848,0.348605,0.945677,0.605405,0.461895,0.00702123
3,0.650546,0.414467,0.877956,0.59157,0.265558,0.566195,0.0079046,0.608079
4,0.268317,0.319089,0.609955,0.902323,0.770762,0.991544,0.0910924,0.792966
5,0.652427,0.765766,0.49364,0.113245,0.230824,0.229971,0.00935943,0.212565
6,0.722184,0.732389,0.502374,0.717295,0.615377,0.982335,0.747881,0.821812
7,0.535124,0.806856,0.251819,0.641313,0.223733,0.743746,0.775601,0.784478
8,0.581566,0.695011,0.654364,0.256979,0.744593,0.110663,0.675994,0.761357
9,0.726788,0.294234,0.429835,0.323258,0.0581465,0.996876,0.776092,0.323629


In [52]:
er.x1 # you can access columns of a parent data frame directly

10-element Vector{Float64}:
 0.9910351570546703
 0.68347262896937
 0.6505455420436681
 0.2683169443047404
 0.6524268881445072
 0.7221844390278147
 0.5351243070160154
 0.5815656647410734
 0.726788246717526
 0.602605613671074

In [53]:
# it prints like a data frame, only the caption is different so that you know the type of the object
ec = eachcol(x)

,x1,x2,x3,x4,x5,x6,x7,x8
,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64
1,0.991035,0.465052,0.380187,0.960458,0.313886,0.132266,0.976021,0.808724
2,0.683473,0.840802,0.0186848,0.348605,0.945677,0.605405,0.461895,0.00702123
3,0.650546,0.414467,0.877956,0.59157,0.265558,0.566195,0.0079046,0.608079
4,0.268317,0.319089,0.609955,0.902323,0.770762,0.991544,0.0910924,0.792966
5,0.652427,0.765766,0.49364,0.113245,0.230824,0.229971,0.00935943,0.212565
6,0.722184,0.732389,0.502374,0.717295,0.615377,0.982335,0.747881,0.821812
7,0.535124,0.806856,0.251819,0.641313,0.223733,0.743746,0.775601,0.784478
8,0.581566,0.695011,0.654364,0.256979,0.744593,0.110663,0.675994,0.761357
9,0.726788,0.294234,0.429835,0.323258,0.0581465,0.996876,0.776092,0.323629


In [54]:
ec.x1 # you can access columns of a parent data frame directly

10-element Vector{Float64}:
 0.9910351570546703
 0.68347262896937
 0.6505455420436681
 0.2683169443047404
 0.6524268881445072
 0.7221844390278147
 0.5351243070160154
 0.5815656647410734
 0.726788246717526
 0.602605613671074

### Transposing

you can transpose a data frame using `permutedims`:

In [55]:
df = DataFrame(reshape(1:12, 3, 4), :auto)

,x1,x2,x3,x4
,Int64,Int64,Int64,Int64
1,1,4,7,10
2,2,5,8,11
3,3,6,9,12


In [56]:
df.names = ["a", "b", "c"]

3-element Vector{String}:
 "a"
 "b"
 "c"

In [57]:
permutedims(df, :names)

,names,a,b,c
,String,Int64,Int64,Int64
1,x1,1,2,3
2,x2,4,5,6
3,x3,7,8,9
4,x4,10,11,12
